<a href="https://colab.research.google.com/github/Ratchadakorn26/GA_project/blob/main/Genetic_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random


In [ ]:
def generate_random_numbers(limit=200):
    while True:
        # สุ่มเลข 3 ตัวในช่วง 1 ถึง 200
        num1 = random.randint(1, limit)
        num2 = random.randint(1, limit)
        num3 = random.randint(1, limit)

        # ตรวจสอบว่าเลขที่สุ่มมา 3 ตัวรวมกันไม่เกิน 200
        if num1 + num2 + num3 <= limit:
            return num1, num2, num3

# ฟังก์ชันสำหรับสุ่ม 100 ชุด
def generate_multiple_random_sets(n=100, limit=200):
    results = []
    for _ in range(n):
        random_numbers = generate_random_numbers(limit)
        results.append(random_numbers)
    return results

# สุ่ม 100 ชุดและแสดงผลลัพธ์
random_sets = generate_multiple_random_sets()

# แสดงผลลัพธ์
for i, random_set in enumerate(random_sets, 1):
    print(f"ชุดที่ {i}: {random_set}, ผลรวม: {sum(random_set)}")

ชุดที่ 1: (5, 100, 2), ผลรวม: 107
ชุดที่ 2: (78, 32, 45), ผลรวม: 155
ชุดที่ 3: (13, 21, 7), ผลรวม: 41
ชุดที่ 4: (52, 51, 32), ผลรวม: 135
ชุดที่ 5: (94, 54, 28), ผลรวม: 176
ชุดที่ 6: (82, 34, 80), ผลรวม: 196
ชุดที่ 7: (8, 54, 32), ผลรวม: 94
ชุดที่ 8: (105, 58, 7), ผลรวม: 170
ชุดที่ 9: (4, 20, 38), ผลรวม: 62
ชุดที่ 10: (42, 55, 42), ผลรวม: 139
ชุดที่ 11: (37, 71, 55), ผลรวม: 163
ชุดที่ 12: (15, 73, 94), ผลรวม: 182
ชุดที่ 13: (66, 28, 95), ผลรวม: 189
ชุดที่ 14: (13, 159, 8), ผลรวม: 180
ชุดที่ 15: (121, 17, 18), ผลรวม: 156
ชุดที่ 16: (27, 116, 53), ผลรวม: 196
ชุดที่ 17: (158, 21, 20), ผลรวม: 199
ชุดที่ 18: (21, 6, 56), ผลรวม: 83
ชุดที่ 19: (10, 41, 9), ผลรวม: 60
ชุดที่ 20: (25, 60, 55), ผลรวม: 140
ชุดที่ 21: (9, 49, 110), ผลรวม: 168
ชุดที่ 22: (49, 43, 73), ผลรวม: 165
ชุดที่ 23: (12, 24, 134), ผลรวม: 170
ชุดที่ 24: (57, 64, 37), ผลรวม: 158
ชุดที่ 25: (19, 82, 24), ผลรวม: 125
ชุดที่ 26: (20, 72, 93), ผลรวม: 185
ชุดที่ 27: (5, 29, 3), ผลรวม: 37
ชุดที่ 28: (43, 69, 1), ผลรวม: 113
ชุดที่ 29: (

In [ ]:
import random
C = 80 + int((200 - 80) * random.random()) # รอบเวลาของไฟจราจร (เช่น 120 วินาที)
X1 = 0.90 # อัตราส่วนของปริมาณยานพาหนะต่อความสามารถในการผ่านทางจากในเมือง
X2 = 0.76 # อัตราส่วนของปริมาณยานพาหนะต่อความสามารถในการผ่านทางจากนอกเมือง
X3 = 0.89 # อัตราส่วนของปริมาณยานพาหนะต่อความสามารถในการผ่านทางจากกังสดาล
# ฟังก์ชันสุ่มเลข 3 ตัวที่รวมกันเท่ากับ C
def generate_random_numbers_sum_to_C():
    # สุ่มสองตัวเลขแรก
    num1 = random.randint(1, C-1)
    num2 = random.randint(1, C - num1)

    # ตัวเลขที่สามต้องทำให้ผลรวมเท่ากับ C
    num3 = C - (num1 + num2)

    return num1, num2, num3

# ฟังก์ชันคำนวณเวลารอคอยเฉลี่ย D
def calculate_waiting_time(C, g, X1):
    return (C * (1 - g / C) ** 2) / (2 * (1 - (g / C) * X1))
def calculate_waiting_time(C, g, X2):
    return (C * (1 - g / C) ** 2) / (2 * (1 - (g / C) * X2))
def calculate_waiting_time(C, g, X3):
    return (C * (1 - g / C) ** 2) / (2 * (1 - (g / C) * X3))

# ฟังก์ชันสำหรับสุ่ม 1000 ชุด และคำนวณเวลารอคอยเฉลี่ยในแต่ละชุด
def generate_multiple_random_sets_and_calculate_waiting(n=1000):
    results = []
    for _ in range(n):
        # สุ่มเลข 3 ตัวที่ผลรวมเป็น C
        num1, num2, num3 = generate_random_numbers_sum_to_C()

        # คำนวณเวลารอคอยเฉลี่ยโดยใช้ตัวเลขสุ่มมาเป็นระยะเวลาไฟเขียว
        D1 = calculate_waiting_time(C, num1, X1)
        D2 = calculate_waiting_time(C, num2, X2)
        D3 = calculate_waiting_time(C, num3, X3)

        # ค่าเฉลี่ยเวลารอคอยของชุดนั้น
        average_waiting_time = (D1 + D2 + D3) / 3
        results.append((num1, num2, num3, average_waiting_time))

    return results

# เรียกใช้ฟังก์ชันสุ่มและคำนวณเวลารอคอย 1000 ชุด
random_sets_with_waiting_time = generate_multiple_random_sets_and_calculate_waiting()

# แสดงผลลัพธ์
for i, (num1, num2, num3, avg_waiting_time) in enumerate(random_sets_with_waiting_time, 1):
    print(f"รอบที่ {i}: เลขสุ่ม = ({num1}, {num2}, {num3}), เวลารอคอยเฉลี่ย = {avg_waiting_time:.2f} วินาที")


รอบที่ 1: เลขสุ่ม = (62, 30, 59), เวลารอคอยเฉลี่ย = 47.22 วินาที
รอบที่ 2: เลขสุ่ม = (103, 27, 21), เวลารอคอยเฉลี่ย = 47.52 วินาที
รอบที่ 3: เลขสุ่ม = (22, 90, 39), เวลารอคอยเฉลี่ย = 46.63 วินาที
รอบที่ 4: เลขสุ่ม = (30, 103, 18), เวลารอคอยเฉลี่ย = 46.80 วินาที
รอบที่ 5: เลขสุ่ม = (95, 34, 22), เวลารอคอยเฉลี่ย = 47.31 วินาที
รอบที่ 6: เลขสุ่ม = (116, 11, 24), เวลารอคอยเฉลี่ย = 48.02 วินาที
รอบที่ 7: เลขสุ่ม = (119, 28, 4), เวลารอคอยเฉลี่ย = 47.75 วินาที
รอบที่ 8: เลขสุ่ม = (121, 2, 28), เวลารอคอยเฉลี่ย = 48.32 วินาที
รอบที่ 9: เลขสุ่ม = (22, 123, 6), เวลารอคอยเฉลี่ย = 47.47 วินาที
รอบที่ 10: เลขสุ่ม = (74, 26, 51), เวลารอคอยเฉลี่ย = 47.33 วินาที
รอบที่ 11: เลขสุ่ม = (23, 114, 14), เวลารอคอยเฉลี่ย = 47.08 วินาที
รอบที่ 12: เลขสุ่ม = (142, 9, 0), เวลารอคอยเฉลี่ย = 49.06 วินาที
รอบที่ 13: เลขสุ่ม = (108, 32, 11), เวลารอคอยเฉลี่ย = 47.49 วินาที
รอบที่ 14: เลขสุ่ม = (119, 7, 25), เวลารอคอยเฉลี่ย = 48.16 วินาที
รอบที่ 15: เลขสุ่ม = (127, 15, 9), เวลารอคอยเฉลี่ย = 48.20 วินาที
รอบที่ 16: เลขส

In [ ]:
import random

C = 185  # รอบเวลาของไฟจราจร (เช่น 120 วินาที)
X1 = 0.90  # อัตราส่วนของปริมาณยานพาหนะต่อความสามารถในการผ่านทางจากในเมือง
X2 = 0.76  # อัตราส่วนของปริมาณยานพาหนะต่อความสามารถในการผ่านทางจากนอกเมือง
X3 = 0.89  # อัตราส่วนของปริมาณยานพาหนะต่อความสามารถในการผ่านทางจากกังสดาล

# ฟังก์ชันสุ่มเลข 3 ตัวที่รวมกันเท่ากับ C
def generate_random_numbers_sum_to_C():
    # สุ่มสองตัวเลขแรก
    num1 = random.randint(60, C-1)
    num2 = random.randint(1, C - num1)

    # ตัวเลขที่สามต้องทำให้ผลรวมเท่ากับ C
    num3 = C - (num1 + num2)

    return num1, num2, num3

# ฟังก์ชันคำนวณเวลารอคอยเฉลี่ย D
def calculate_waiting_time(C, g, X):
    return (C * (1 - g / C) ** 2) / (2 * (1 - (g / C) * X))

# ฟังก์ชันสำหรับสุ่ม 1000 ชุด และคำนวณเวลารอคอยเฉลี่ยในแต่ละชุด
def generate_multiple_random_sets_and_calculate_waiting(n=1000):
    results = []
    for _ in range(n):
        # สุ่มเลข 3 ตัวที่ผลรวมเป็น C
        num1, num2, num3 = generate_random_numbers_sum_to_C()

        # คำนวณเวลารอคอยเฉลี่ยโดยใช้ตัวเลขสุ่มมาเป็นระยะเวลาไฟเขียว
        D1 = calculate_waiting_time(C, num1, X1)
        D2 = calculate_waiting_time(C, num2, X2)
        D3 = calculate_waiting_time(C, num3, X3)

        # ค่าเฉลี่ยเวลารอคอยของชุดนั้น
        average_waiting_time = (D1 + D2 + D3) / 3
        results.append((num1, num2, num3, average_waiting_time))

    return results

# เรียกใช้ฟังก์ชันสุ่มและคำนวณเวลารอคอย 1000 ชุด
random_sets_with_waiting_time = generate_multiple_random_sets_and_calculate_waiting()

# หาค่าที่มีเวลารอคอยเฉลี่ยน้อยที่สุด
best_set = min(random_sets_with_waiting_time, key=lambda x: x[3])

# แสดงผลลัพธ์ที่มีเวลารอคอยน้อยที่สุด
for i, (num1, num2, num3, avg_waiting_time) in enumerate(random_sets_with_waiting_time, 1):
    print(f"รอบที่ {i}: เลขสุ่ม = ({num1}, {num2}, {num3}), เวลารอคอยเฉลี่ย = {avg_waiting_time:.2f} วินาที")
num1, num2, num3, min_avg_waiting_time = best_set
print(f"ชุดที่มีเวลารอคอยน้อยที่สุด: เลขสุ่ม = ({num1}, {num2}, {num3}), เวลารอคอยเฉลี่ย = {min_avg_waiting_time:.2f} วินาที")


รอบที่ 1: เลขสุ่ม = (117, 64, 4), เวลารอคอยเฉลี่ย = 57.66 วินาที
รอบที่ 2: เลขสุ่ม = (165, 14, 6), เวลารอคอยเฉลี่ย = 59.50 วินาที
รอบที่ 3: เลขสุ่ม = (161, 23, 1), เวลารอคอยเฉลี่ย = 59.15 วินาที
รอบที่ 4: เลขสุ่ม = (173, 11, 1), เวลารอคอยเฉลี่ย = 60.03 วินาที
รอบที่ 5: เลขสุ่ม = (62, 27, 96), เวลารอคอยเฉลี่ย = 58.07 วินาที
รอบที่ 6: เลขสุ่ม = (148, 17, 20), เวลารอคอยเฉลี่ย = 58.88 วินาที
รอบที่ 7: เลขสุ่ม = (112, 28, 45), เวลารอคอยเฉลี่ย = 58.18 วินาที
รอบที่ 8: เลขสุ่ม = (88, 31, 66), เวลารอคอยเฉลี่ย = 58.00 วินาที
รอบที่ 9: เลขสุ่ม = (113, 43, 29), เวลารอคอยเฉลี่ย = 57.91 วินาที
รอบที่ 10: เลขสุ่ม = (76, 105, 4), เวลารอคอยเฉลี่ย = 57.22 วินาที
รอบที่ 11: เลขสุ่ม = (77, 38, 70), เวลารอคอยเฉลี่ย = 57.83 วินาที
รอบที่ 12: เลขสุ่ม = (113, 1, 71), เวลารอคอยเฉลี่ย = 58.78 วินาที
รอบที่ 13: เลขสุ่ม = (107, 17, 61), เวลารอคอยเฉลี่ย = 58.37 วินาที
รอบที่ 14: เลขสุ่ม = (112, 31, 42), เวลารอคอยเฉลี่ย = 58.12 วินาที
รอบที่ 15: เลขสุ่ม = (91, 70, 24), เวลารอคอยเฉลี่ย = 57.41 วินาที
รอบที่ 16: เลข

In [ ]:
import random
C = 185 # รอบเวลาของไฟจราจร (เช่น 120 วินาที)
X1 = 0.90 # อัตราส่วนของปริมาณยานพาหนะต่อความสามารถในการผ่านทางจากในเมือง
X2 = 0.76 # อัตราส่วนของปริมาณยานพาหนะต่อความสามารถในการผ่านทางจากนอกเมือง
X3 = 0.89 # อัตราส่วนของปริมาณยานพาหนะต่อความสามารถในการผ่านทางจากกังสดาล

# จำนวนโครโมโซมในประชากรเริ่มต้น
population_size = 100
generations = 1000

# ฟังก์ชันสุ่มโครโมโซม (ชุดค่าไฟเขียว) ที่รวมกันเท่ากับ C
def generate_chromosome(C):
    num1 = random.randint(1, C-1)
    num2 = random.randint(1, C - num1)
    num3 = C - (num1 + num2)
    return [num1, num2, num3]

# ฟังก์ชันคำนวณ fitness (เวลารอคอยเฉลี่ย)
def calculate_fitness(chromosome, C, X1, X2, X3):
    num1, num2, num3 = chromosome
    D1 = calculate_waiting_time(C, num1, X1)
    D2 = calculate_waiting_time(C, num2, X2)
    D3 = calculate_waiting_time(C, num3, X3)
    return (D1 + D2 + D3) / 3  # ค่า fitness คือค่าเฉลี่ยเวลารอคอย (ยิ่งน้อยยิ่งดี)

# ฟังก์ชันเลือกโครโมโซมโดยใช้ roulette wheel selection
def select_chromosome(population, fitness_scores):
    total_fitness = sum(fitness_scores)
    selection_probs = [fitness / total_fitness for fitness in fitness_scores]
    return population[random.choices(range(len(population)), weights=selection_probs, k=1)[0]]

# ฟังก์ชัน crossover ระหว่างโครโมโซม 2 ตัว
def crossover(parent1, parent2):
    crossover_point = random.randint(0, 2)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

# ฟังก์ชันกลายพันธุ์ (การสุ่มเปลี่ยนแปลงค่า)
def mutate(chromosome, C):
    mutation_point = random.randint(0, 2)
    chromosome[mutation_point] = random.randint(1, C - sum(chromosome[:mutation_point]))
    return chromosome

# ฟังก์ชันหลักสำหรับการทำ Genetic Algorithm
def genetic_algorithm(C, X1, X2, X3, population_size, generations):
    # สร้างประชากรเริ่มต้น
    population = [generate_chromosome(C) for _ in range(population_size)]

    for generation in range(generations):
        # คำนวณ fitness ของแต่ละโครโมโซม
        fitness_scores = [calculate_fitness(chromosome, C, X1, X2, X3) for chromosome in population]

        # สร้างประชากรใหม่
        new_population = []
        for _ in range(population_size // 2):
            # คัดเลือกพ่อแม่
            parent1 = select_chromosome(population, fitness_scores)
            parent2 = select_chromosome(population, fitness_scores)

            # ทำ crossover เพื่อสร้างลูก
            child1, child2 = crossover(parent1, parent2)

            # กลายพันธุ์
            child1 = mutate(child1, C)
            child2 = mutate(child2, C)

            # เพิ่มลูกเข้าสู่ประชากรใหม่
            new_population.extend([child1, child2])

        population = new_population  # ประชากรรุ่นใหม่

        # แสดงผลลัพธ์ระหว่างกระบวนการ
        best_fitness = min(fitness_scores)
        print(f"Generation {generation + 1}: Best Fitness = {best_fitness:.2f}")

    # หาคำตอบที่ดีที่สุด
    best_chromosome = population[fitness_scores.index(min(fitness_scores))]
    return best_chromosome

# เรียกใช้ Genetic Algorithm
best_solution = genetic_algorithm(C, X1, X2, X3, population_size, generations)
print("โครโมโซมที่ดีที่สุด:", best_solution)


ValueError: empty range for randrange() (1, -32, -33)

In [ ]:
import random

# จำนวนโครโมโซมในประชากรเริ่มต้น
population_size = 100
generations = 1000

# ฟังก์ชันสุ่มโครโมโซม (ชุดค่าไฟเขียว) ที่รวมกันเท่ากับ C
def generate_chromosome(C):
    num1 = random.randint(1, C-1)
    num2 = random.randint(1, C - num1)
    num3 = C - (num1 + num2)
    return [num1, num2, num3]

# ฟังก์ชันคำนวณเวลารอคอยเฉลี่ย (average waiting time)
def calculate_waiting_time(C, g, X):
    return (C * (1 - g / C) ** 2) / (2 * (1 - (g / C) * X))

# ฟังก์ชันคำนวณ Fitness โดยใช้ค่า average_waiting_time
def calculate_fitness(chromosome, C, X1, X2, X3):
    num1, num2, num3 = chromosome
    D1 = calculate_waiting_time(C, num1, X1)
    D2 = calculate_waiting_time(C, num2, X2)
    D3 = calculate_waiting_time(C, num3, X3)
    average_waiting_time = (D1 + D2 + D3) / 3
    return 1 / (1 + average_waiting_time)  # ยิ่งค่า fitness สูง ยิ่งดี

# ฟังก์ชันเลือกโครโมโซมโดยใช้ roulette wheel selection
def select_chromosome(population, fitness_scores):
    total_fitness = sum(fitness_scores)
    selection_probs = [fitness / total_fitness for fitness in fitness_scores]
    return population[random.choices(range(len(population)), weights=selection_probs, k=1)[0]]

# ฟังก์ชัน crossover ระหว่างโครโมโซม 2 ตัว
def crossover(parent1, parent2):
    crossover_point = random.randint(0, 2)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

# ฟังก์ชันกลายพันธุ์ (การสุ่มเปลี่ยนแปลงค่า)
def mutate(chromosome, C):
    mutation_point = random.randint(0, 2)
    chromosome[mutation_point] = random.randint(1, C - sum(chromosome[:mutation_point]))
    return chromosome

# ฟังก์ชันหลักสำหรับการทำ Genetic Algorithm
def genetic_algorithm(C, X1, X2, X3, population_size, generations):
    # สร้างประชากรเริ่มต้น
    population = [generate_chromosome(C) for _ in range(population_size)]

    for generation in range(generations):
        # คำนวณ fitness ของแต่ละโครโมโซม
        fitness_scores = [calculate_fitness(chromosome, C, X1, X2, X3) for chromosome in population]

        # สร้างประชากรใหม่
        new_population = []
        for _ in range(population_size // 2):
            # คัดเลือกพ่อแม่
            parent1 = select_chromosome(population, fitness_scores)
            parent2 = select_chromosome(population, fitness_scores)

            # ทำ crossover เพื่อสร้างลูก
            child1, child2 = crossover(parent1, parent2)

            # กลายพันธุ์
            child1 = mutate(child1, C)
            child2 = mutate(child2, C)

            # เพิ่มลูกเข้าสู่ประชากรใหม่
            new_population.extend([child1, child2])

        population = new_population  # ประชากรรุ่นใหม่

        # แสดงผลลัพธ์ระหว่างกระบวนการ
        best_fitness = max(fitness_scores)
        print(f"Generation {generation + 1}: Best Fitness = {best_fitness:.6f}")

    # หาคำตอบที่ดีที่สุด
    best_chromosome = population[fitness_scores.index(max(fitness_scores))]
    return best_chromosome

# ค่าเริ่มต้น
C = 80 + int((200 - 80) * random.random())  # รอบเวลาของไฟจราจร (เช่น 120 วินาที)
X1 = 0.90  # อัตราส่วนของปริมาณยานพาหนะต่อความสามารถในการผ่านทางจากในเมือง
X2 = 0.76  # อัตราส่วนของปริมาณยานพาหนะต่อความสามารถในการผ่านทางจากนอกเมือง
X3 = 0.89  # อัตราส่วนของปริมาณยานพาหนะต่อความสามารถในการผ่านทางจากกังสดาล

# เรียกใช้ Genetic Algorithm วนลูป 1000 รอบ
best_solution = genetic_algorithm(C, X1, X2, X3, population_size, generations)
print("โครโมโซมที่ดีที่สุด:", best_solution)


ValueError: empty range for randrange() (1, -44, -45)

In [ ]:
from random import randint as rnd
import random
import copy
import math


def genR(len_chromosome):
    return [round(random.random(), 4) for _ in  range(len_chromosome)]

def gen_Algorithm(crsOver_rate, perMut_rate):
    temp_Round = []
    # gen Chromosome
    Round_number = 0  # temp round
    Chromosome = [[rnd(1, 41), rnd(1, 41), rnd(1, 41), rnd(1, 41)] for _ in range(6)]  # init Chromosome

    while True:
        Round_number += 1  # tmp round
        # Evaluation
        F_obj = []
        for i_eva, n in zip(Chromosome, range(len(Chromosome))):
            F_obj.append(abs(2 * i_eva[0] + 4 * i_eva[1] + 3 * i_eva[2] - 2 * i_eva[3] - 41))
            if F_obj[n] == 0:
                Best_Chromosome = copy.deepcopy(i_eva)
                break
        if min(F_obj) != 0:
            print("Round : {} Min : {} - Population : {}".format(Round_number, min(F_obj), Chromosome))
        else:
            print('\n')
            print("-"*40)
            temp_Round.append(Round_number)
            print("Round : {} Min : {} - Population : {}".format(Round_number, min(F_obj), Chromosome))
            print("Best Chromosome : ", Best_Chromosome)
            print("F_obj : ", min(F_obj))
            print("-"*40)
            print('\n')
            break

        # Selection
        Fitness_select = []
        Total_Fitness = 0
        Prob_select = []
        Cumulative = 0
        Prob_Cumulative = []
        for i_select in range(len(F_obj)):
            Fitness_select.append(round(1 / (1 + F_obj[i_select]), 4))
            Total_Fitness += Fitness_select[i_select]
        for i_prob in range(len(F_obj)):
            Prob_select.append(round(Fitness_select[i_prob] / Total_Fitness, 4))
            Cumulative += Prob_select[i_prob]
            Prob_Cumulative.append(round(Cumulative, 4))

        # gen R for new Chromosome
        R_newChr = genR(len(Chromosome))

        # Gen new Chromosome
        New_chromosome = []
        for R_round in range(len(Chromosome)):
            for C_check in range(len(Chromosome)):
                if R_newChr[R_round] <= Prob_Cumulative[C_check]:
                    New_chromosome.append(Chromosome[C_check])
                    break
        # gen R for Parent
        R_parent = genR(len(New_chromosome))
        # gen Parent
        crs_Rate = round(crsOver_rate / 100, 4)
        Parent = [New_chromosome[i_crs] for i_crs in range(len(New_chromosome)) if R_parent[i_crs] <= crs_Rate]
        Cut_Points = [rnd(1, 3) for _ in range(len(Parent))]
        Parent_Idx = [i_crs for i_crs in range(len(New_chromosome)) if R_parent[i_crs] <= crs_Rate]

        # #beforeCrossoverChromosome = copy.deepcopy(New_chromosome)
        # crossOver
        if len(Parent) >= 1:
            Parent_crs = copy.deepcopy(Parent)
            for n, cut_point in zip(range(len(Parent)), Cut_Points):
                del Parent_crs[n][cut_point:]
                if n != len(Parent) - 1:  # if it's not last chromosome
                    for index, val in enumerate(Parent[n + 1]):
                        if index >= cut_point:
                            Parent_crs[n].append(val)
                else:  # if it's last chromosome
                    for index, val in enumerate(Parent[0]):
                        if index >= cut_point:
                            Parent_crs[n].append(val)
            # assign parent chromosome back to New_Chromosome
            for lis, n in zip(Parent_Idx, range(len(Parent_Idx))):
                del New_chromosome[lis][:]
                for _, val in enumerate(Parent_crs[n]):
                    New_chromosome[lis].append(val)

            # #beforeMutationChromosome = copy.deepcopy(New_chromosome)
            # Mutation
            Permutation_rate = round(perMut_rate / 100, 4)
            Total_gen = len(New_chromosome) * len(New_chromosome[0])  # total len of gen
            Num_mutation = math.floor(Permutation_rate * Total_gen)
            PosChro = []  # Position in chromosome
            ValLis = []  # value of mutated
            PosLis = []  # Position of List(Chromosome)
            PosInLis = []  # Position in PosLis
            for n in range(Num_mutation):
                PosChro.append(rnd(0, Total_gen - 1))
                ValLis.append(rnd(1, 41))
                PosLis.append(PosChro[n] // 4)
                PosInLis.append(PosChro[n] % 4)
                New_chromosome[PosLis[n]].insert(PosInLis[n], ValLis[n])  # insert(inx, ele)
                del New_chromosome[PosLis[n]][PosInLis[n] + 1]
        Chromosome = copy.deepcopy(New_chromosome)


if __name__ == '__main__':
    gen_Algorithm(30, 20)  # crossOver_Rate, mutation_Rate

Round : 1 Min : 1 - Population : [[4, 30, 41, 1], [40, 8, 28, 27], [2, 8, 25, 18], [19, 8, 37, 15], [1, 6, 12, 10], [22, 34, 5, 21]]
Round : 2 Min : 11 - Population : [[1, 6, 28, 40], [1, 6, 28, 40], [1, 6, 28, 40], [1, 6, 28, 40], [1, 6, 28, 40], [1, 6, 28, 40]]
Round : 3 Min : 130 - Population : [[25, 14, 29, 11], [25, 14, 29, 11], [25, 14, 29, 11], [25, 14, 29, 11], [25, 14, 29, 11], [25, 14, 29, 11]]
Round : 4 Min : 202 - Population : [[25, 30, 29, 7], [25, 30, 29, 7], [25, 30, 29, 7], [25, 30, 29, 7], [25, 30, 29, 7], [25, 30, 29, 7]]
Round : 5 Min : 153 - Population : [[26, 30, 12, 7], [26, 30, 12, 7], [26, 30, 12, 7], [26, 30, 12, 7], [26, 30, 12, 7], [26, 30, 12, 7]]
Round : 6 Min : 159 - Population : [[30, 28, 14, 7], [30, 28, 14, 7], [30, 28, 14, 7], [30, 28, 14, 7], [30, 28, 14, 7], [30, 28, 14, 7]]
Round : 7 Min : 159 - Population : [[30, 28, 14, 7], [30, 28, 14, 7], [30, 28, 14, 7], [30, 28, 14, 7], [30, 28, 14, 7], [30, 28, 14, 7]]
Round : 8 Min : 159 - Population : [[30,

###ตัวอย่างการใช้ Genetic Algorithm

In [1]:
from random import randint as rnd
import random
import copy
import math

""" SOLVING a, b, c, d in equation 2a+4b+3c-2d=41 """

def genR(len_chromosome):
    return [round(random.random(), 3) for _ in  range(len_chromosome)]

def gen_Algorithm(crsOver_rate, perMut_rate):
    temp_Round = []
    # gen Chromosome
    Round_number = 0  # temp round
    Chromosome = [[rnd(1, 41), rnd(1, 41), rnd(1, 41), rnd(1, 41)] for _ in range(6)]  # init Chromosome

    while True:
        Round_number += 1  # tmp round
        # Evaluation
        F_obj = []
        for i_eva, n in zip(Chromosome, range(len(Chromosome))):
            F_obj.append(abs(2 * i_eva[0] + 4 * i_eva[1] + 3 * i_eva[2] - 2 * i_eva[3] - 41))
            if F_obj[n] == 0:
                Best_Chromosome = copy.deepcopy(i_eva)
                break
        if min(F_obj) != 0:
            print("Round : {} Min : {} - Population : {}".format(Round_number, min(F_obj), Chromosome))
        else:
            print('\n')
            print("-"*40)
            temp_Round.append(Round_number)
            print("Round : {} Min : {} - Population : {}".format(Round_number, min(F_obj), Chromosome))
            print("Best Chromosome : ", Best_Chromosome)
            print("F_obj : ", min(F_obj))
            print("-"*40)
            print('\n')
            break

        # Selection
        Fitness_select = []
        Total_Fitness = 0
        Prob_select = []
        Cumulative = 0
        Prob_Cumulative = []
        for i_select in range(len(F_obj)):
            Fitness_select.append(round(1 / (1 + F_obj[i_select]), 4))
            Total_Fitness += Fitness_select[i_select]
        for i_prob in range(len(F_obj)):
            Prob_select.append(round(Fitness_select[i_prob] / Total_Fitness, 4))
            Cumulative += Prob_select[i_prob]
            Prob_Cumulative.append(round(Cumulative, 4))

        # gen R for new Chromosome
        R_newChr = genR(len(Chromosome))

        # Gen new Chromosome
        New_chromosome = []
        for R_round in range(len(Chromosome)):
            for C_check in range(len(Chromosome)):
                if R_newChr[R_round] <= Prob_Cumulative[C_check]:
                    New_chromosome.append(Chromosome[C_check])
                    break
        # gen R for Parent
        R_parent = genR(len(New_chromosome))
        # gen Parent
        crs_Rate = round(crsOver_rate / 100, 4)
        Parent = [New_chromosome[i_crs] for i_crs in range(len(New_chromosome)) if R_parent[i_crs] <= crs_Rate]
        Cut_Points = [rnd(1, 3) for _ in range(len(Parent))]
        Parent_Idx = [i_crs for i_crs in range(len(New_chromosome)) if R_parent[i_crs] <= crs_Rate]

        # #beforeCrossoverChromosome = copy.deepcopy(New_chromosome)
        # crossOver
        if len(Parent) >= 1:
            Parent_crs = copy.deepcopy(Parent)
            for n, cut_point in zip(range(len(Parent)), Cut_Points):
                del Parent_crs[n][cut_point:]
                if n != len(Parent) - 1:  # if it's not last chromosome
                    for index, val in enumerate(Parent[n + 1]):
                        if index >= cut_point:
                            Parent_crs[n].append(val)
                else:  # if it's last chromosome
                    for index, val in enumerate(Parent[0]):
                        if index >= cut_point:
                            Parent_crs[n].append(val)
            # assign parent chromosome back to New_Chromosome
            for lis, n in zip(Parent_Idx, range(len(Parent_Idx))):
                del New_chromosome[lis][:]
                for _, val in enumerate(Parent_crs[n]):
                    New_chromosome[lis].append(val)

            # #beforeMutationChromosome = copy.deepcopy(New_chromosome)
            # Mutation
            Permutation_rate = round(perMut_rate / 100, 4)
            Total_gen = len(New_chromosome) * len(New_chromosome[0])  # total len of gen
            Num_mutation = math.floor(Permutation_rate * Total_gen)
            PosChro = []  # Position in chromosome
            ValLis = []  # value of mutated
            PosLis = []  # Position of List(Chromosome)
            PosInLis = []  # Position in PosLis
            for n in range(Num_mutation):
                PosChro.append(rnd(0, Total_gen - 1))
                ValLis.append(rnd(1, 41))
                PosLis.append(PosChro[n] // 4)
                PosInLis.append(PosChro[n] % 4)
                New_chromosome[PosLis[n]].insert(PosInLis[n], ValLis[n])  # insert(inx, ele)
                del New_chromosome[PosLis[n]][PosInLis[n] + 1]
        Chromosome = copy.deepcopy(New_chromosome)


if __name__ == '__main__':
    gen_Algorithm(30, 20)  # crossOver_Rate, mutation_Rate

Round : 1 Min : 36 - Population : [[10, 17, 23, 40], [5, 15, 41, 20], [28, 7, 20, 14], [13, 40, 39, 29], [29, 35, 15, 1], [24, 12, 33, 33]]
Round : 2 Min : 6 - Population : [[28, 7, 20, 14], [5, 17, 23, 40], [10, 18, 5, 30], [24, 13, 41, 20], [24, 13, 41, 20], [10, 18, 5, 30]]
Round : 3 Min : 6 - Population : [[10, 18, 5, 36], [10, 18, 5, 36], [10, 18, 5, 36], [10, 18, 5, 36], [10, 18, 5, 36], [10, 18, 5, 36]]
Round : 4 Min : 37 - Population : [[10, 9, 14, 10], [10, 9, 14, 10], [10, 9, 14, 10], [10, 9, 14, 10], [10, 9, 14, 10], [10, 9, 14, 10]]
Round : 5 Min : 55 - Population : [[35, 9, 18, 32], [35, 9, 18, 32], [35, 9, 18, 32], [35, 9, 18, 32], [35, 9, 18, 32], [35, 9, 18, 32]]
Round : 6 Min : 7 - Population : [[2, 9, 18, 23], [2, 9, 18, 23], [2, 9, 18, 23], [2, 9, 18, 23], [2, 9, 18, 23], [2, 9, 18, 23]]
Round : 7 Min : 86 - Population : [[3, 24, 23, 22], [3, 24, 23, 22], [3, 24, 23, 22], [3, 24, 23, 22], [3, 24, 23, 22], [3, 24, 23, 22]]
Round : 8 Min : 150 - Population : [[14, 41, 

In [27]:
import numpy as np
import random

# ค่าคงที่
CYCLE_TIME = 89  # รอบเวลาทั้งหมด
q_values = [1802, 1497, 542]  # อัตราการไหลของรถแต่ละเฟส
x_values = [0.263, 0.233, 0.17]  # Degree of Saturation

# ฟังก์ชันคำนวณค่า d_i ตามสูตร
def calculate_wait_time(green_times):
    total_wait_time = 0

    for i in range(3):  # 3 เฟส
        g_i = green_times[i]
        q_i = q_values[i]
        x_i = x_values[i]

        term1 = (CYCLE_TIME / 2) * ((1 - (g_i / CYCLE_TIME)) ** 2) / (1 - (g_i / CYCLE_TIME) * x_i)
        term2 = (x_i ** 2) / (2 * q_i * (1 - x_i))
        term3 = ((CYCLE_TIME / q_i) ** 0.5) * (x_i ** (2 + (5 * g_i / CYCLE_TIME)))

        d_i = term1 + term2 - term3
        total_wait_time += d_i  # รวมเวลารอของทุกเฟส

    return total_wait_time  # ยิ่งน้อยยิ่งดี (Minimization Problem)

# ฟังก์ชันความเหมาะสม (Fitness Function)
def fitness_function(green_times):
    wait_time = calculate_wait_time(green_times)
    return 1 / (wait_time + 1)  # แปลงเป็น Fitness (ยิ่งมากยิ่งดี)

# ฟังก์ชันสร้างประชากรเริ่มต้น
def initialize_population(pop_size, num_phases):
    population = []
    for _ in range(pop_size):
        green_times = np.random.randint(5, CYCLE_TIME - 10, num_phases)  # สุ่มค่าเริ่มต้น
        green_times = (green_times / green_times.sum()) * CYCLE_TIME  # ปรับให้รวมกันเป็น 89
        green_times = np.round(green_times).astype(int)
        green_times[-1] = CYCLE_TIME - green_times[:-1].sum()  # ป้องกัน error
        population.append(green_times)
    return np.array(population)

# Genetic Algorithm
def genetic_algorithm(num_phases=3, pop_size=50, generations=100, mutation_rate=0.1):
    population = initialize_population(pop_size, num_phases)

    for gen in range(generations):
        fitness_scores = np.array([fitness_function(ind) for ind in population])
        new_population = []

        for _ in range(pop_size // 2):
            parent1 = population[np.argmax(fitness_scores)]  # เอาตัวที่ดีที่สุดมาใช้
            parent2 = population[np.random.randint(len(population))]
            child1, child2 = (parent1 + parent2) // 2, (parent2 + parent1) // 2
            new_population.extend([child1, child2])

        population = np.array(new_population)

        if gen % 10 == 0:
            best_idx = np.argmax(fitness_scores)
            print(f"Generation {gen}: Best Green Times = {population[best_idx]}, Fitness = {fitness_scores[best_idx]:.4f}")

    best_idx = np.argmax(fitness_scores)
    return population[best_idx]
# เรียกใช้งาน Genetic Algorithm
best_green_times = genetic_algorithm()
print("\n🔹 Optimal Green Times:", best_green_times)

Generation 0: Best Green Times = [27 27 35], Fitness = 0.0153
Generation 10: Best Green Times = [24 32 31], Fitness = 0.0152
Generation 20: Best Green Times = [24 32 31], Fitness = 0.0152
Generation 30: Best Green Times = [24 32 30], Fitness = 0.0149
Generation 40: Best Green Times = [24 32 30], Fitness = 0.0149
Generation 50: Best Green Times = [24 32 30], Fitness = 0.0149
Generation 60: Best Green Times = [24 32 30], Fitness = 0.0149
Generation 70: Best Green Times = [24 32 30], Fitness = 0.0149
Generation 80: Best Green Times = [24 32 30], Fitness = 0.0149
Generation 90: Best Green Times = [24 32 30], Fitness = 0.0149

🔹 Optimal Green Times: [24 32 30]


In [46]:
import numpy as np
import random

# ค่าคงที่
CYCLE_TIME = 89  # รอบเวลาทั้งหมด
q_values = [1802, 1497, 542]  # อัตราการไหลของรถแต่ละเฟส (คัน/ชั่วโมง)
x_values = [0.263, 0.233, 0.17]  # Degree of Saturation

# ฟังก์ชันคำนวณค่า d_i ตามสูตรที่ให้มา
def calculate_wait_time(green_times):
    total_wait_time = 0

    for i in range(3):  # 3 เฟส (3 ทิศทาง)
        g_i = green_times[i]
        q_i = q_values[i]
        x_i = x_values[i]

        term1 = (CYCLE_TIME / 2) * ((1 - (g_i / CYCLE_TIME)) ** 2) / (1 - (g_i / CYCLE_TIME) * x_i)
        term2 = (x_i ** 2) / (2 * q_i * (1 - x_i))
        term3 = ((CYCLE_TIME / q_i) ** 0.5) * (x_i ** (2 + (5 * g_i / CYCLE_TIME)))

        d_i = term1 + term2 - term3
        total_wait_time += d_i  # รวมเวลารอของทุกเฟส

    return total_wait_time  # ค่าที่น้อยที่สุดคือคำตอบที่ดีที่สุด

# ฟังก์ชันความเหมาะสม (Fitness Function)
def fitness_function(green_times):
    wait_time = calculate_wait_time(green_times)
    return 1 / (wait_time + 1)  # เปลี่ยนเป็นค่าฟิตเนส (ยิ่งมากยิ่งดี)

# ฟังก์ชันสร้างประชากรเริ่มต้น
def initialize_population(pop_size, num_phases):
    population = []
    for _ in range(pop_size):
        green_times = np.random.randint(5, CYCLE_TIME - 10, num_phases)  # สุ่มค่าเริ่มต้น
        green_times = (green_times / green_times.sum()) * CYCLE_TIME  # ปรับให้รวมกันเป็น 89
        green_times = np.round(green_times).astype(int)
        green_times[-1] = CYCLE_TIME - green_times[:-1].sum()  # ป้องกัน error
        population.append(green_times)
    return np.array(population)

# Genetic Algorithm
def genetic_algorithm(num_phases=3, pop_size=50, mutation_rate=0.1, max_no_change=30):
    population = initialize_population(pop_size, num_phases)

    best_fitness = -float('inf')
    best_solution = None
    no_change_count = 0

    generation = 0
    while no_change_count < max_no_change:
        fitness_scores = np.array([fitness_function(ind) for ind in population])
        best_idx = np.argmax(fitness_scores)
        best_current_fitness = fitness_scores[best_idx]
        best_current_solution = population[best_idx]

        print(f"Generation {generation}: Best Green Times = {best_current_solution}, Fitness = {best_current_fitness:.4f}")

        # ตรวจสอบว่าค่าฟิตเนสไม่เปลี่ยนแปลง
        if best_current_fitness > best_fitness:
            best_fitness = best_current_fitness
            best_solution = best_current_solution
            no_change_count = 0  # รีเซ็ตตัวนับเมื่อมีการพัฒนา
        else:
            no_change_count += 1  # นับถ้าไม่มีการเปลี่ยนแปลง

        # สร้างประชากรใหม่
        new_population = []
        for _ in range(pop_size // 2):
            parent1 = population[np.argmax(fitness_scores)]  # เลือกตัวที่ดีที่สุด
            parent2 = population[np.random.randint(len(population))]
            child1, child2 = (parent1 + parent2) // 2, (parent2 + parent1) // 2
            new_population.extend([child1, child2])

        population = np.array(new_population)
        generation += 1

    print("\n✅ Optimal Green Times Found:")
    print(f"🔹 Green Times: {best_solution}")
    print(f"🔹 Fitness Score: {best_fitness:.4f}")

    return best_solution

# เรียกใช้งาน Genetic Algorithm
best_green_times = genetic_algorithm()

Generation 0: Best Green Times = [31 28 30], Fitness = 0.0154
Generation 1: Best Green Times = [29 26 34], Fitness = 0.0153
Generation 2: Best Green Times = [28 29 31], Fitness = 0.0152
Generation 3: Best Green Times = [28 29 31], Fitness = 0.0152
Generation 4: Best Green Times = [28 29 31], Fitness = 0.0152
Generation 5: Best Green Times = [28 29 31], Fitness = 0.0152
Generation 6: Best Green Times = [28 29 31], Fitness = 0.0152
Generation 7: Best Green Times = [28 29 31], Fitness = 0.0152
Generation 8: Best Green Times = [28 29 31], Fitness = 0.0152
Generation 9: Best Green Times = [28 29 31], Fitness = 0.0152
Generation 10: Best Green Times = [28 29 31], Fitness = 0.0152
Generation 11: Best Green Times = [28 29 31], Fitness = 0.0152
Generation 12: Best Green Times = [28 29 31], Fitness = 0.0152
Generation 13: Best Green Times = [28 29 31], Fitness = 0.0152
Generation 14: Best Green Times = [28 29 31], Fitness = 0.0152
Generation 15: Best Green Times = [28 29 31], Fitness = 0.0152
Ge

In [2]:
import numpy as np
import random

# ค่าคงที่
CYCLE_TIME = 89  # รอบเวลาทั้งหมด
q_values = [1802, 1497, 542]  # อัตราการไหลของรถแต่ละเฟส (คัน/ชั่วโมง)
x_values = [0.263, 0.233, 0.17]  # Degree of Saturation

# ฟังก์ชันคำนวณค่า d_i ตามสูตรที่ให้มา
def calculate_wait_time(green_times):
    total_wait_time = 0

    for i in range(3):  # 3 เฟส
        g_i = green_times[i]
        q_i = q_values[i]
        x_i = x_values[i]

        term1 = (CYCLE_TIME / 2) * ((1 - (g_i / CYCLE_TIME)) ** 2) / (1 - (g_i / CYCLE_TIME) * x_i)
        term2 = (x_i ** 2) / (2 * q_i * (1 - x_i))
        term3 = ((CYCLE_TIME / q_i) ** 0.5) * (x_i ** (2 + (5 * g_i / CYCLE_TIME)))

        d_i = term1 + term2 - term3
        total_wait_time += d_i

    return total_wait_time  # ยิ่งน้อยยิ่งดี

# ฟังก์ชันความเหมาะสม (Fitness Function)
def fitness_function(green_times):
    wait_time = calculate_wait_time(green_times)
    return 1 / (wait_time + 1)  # แปลงเป็นค่าฟิตเนส (ยิ่งมากยิ่งดี)

# ฟังก์ชันสร้างประชากรเริ่มต้น (10 โครโมโซม)
def initialize_population(pop_size=10, num_phases=3):
    population = []
    for _ in range(pop_size):
        green_times = np.random.randint(5, CYCLE_TIME - 10, num_phases)  # สุ่มค่าเริ่มต้น
        green_times = (green_times / green_times.sum()) * CYCLE_TIME  # ปรับให้รวมกันเป็น 89
        green_times = np.round(green_times).astype(int)
        green_times[-1] = CYCLE_TIME - green_times[:-1].sum()  # ป้องกัน error
        population.append(green_times)
    return np.array(population)

# Genetic Algorithm (GA) ที่หยุดเมื่อค่าฟิตเนสไม่เปลี่ยนแปลงต่อเนื่อง 30 รุ่น
def genetic_algorithm(num_phases=3, pop_size=10, mutation_rate=0.1, max_stagnant_generations=30):
    population = initialize_population(pop_size, num_phases)

    best_fitness = -float('inf')
    best_solution = None
    stagnant_generations = 0  # นับจำนวนรอบที่ค่าฟิตเนสไม่เปลี่ยน

    generation = 0
    while stagnant_generations < max_stagnant_generations:
        fitness_scores = np.array([fitness_function(ind) for ind in population])
        best_idx = np.argmax(fitness_scores)
        best_current_fitness = fitness_scores[best_idx]
        best_current_solution = population[best_idx]

        print(f"Generation {generation}: Best Green Times = {best_current_solution}, Fitness = {best_current_fitness:.6f}")

        # เช็คว่ามีการพัฒนาหรือไม่
        if best_current_fitness > best_fitness:
            best_fitness = best_current_fitness
            best_solution = best_current_solution
            stagnant_generations = 0  # รีเซ็ตตัวนับเมื่อมีการพัฒนา
        else:
            stagnant_generations += 1  # เพิ่มตัวนับเมื่อไม่มีการพัฒนา

        # สร้างประชากรใหม่ (เลือกพ่อแม่ + crossover)
        new_population = []
        for _ in range(pop_size // 2):
            parent1 = population[np.argmax(fitness_scores)]  # ตัวที่ดีที่สุด
            parent2 = population[np.random.randint(len(population))]
            child1, child2 = (parent1 + parent2) // 2, (parent2 + parent1) // 2

            # Mutation (สุ่มเปลี่ยนแปลงเล็กน้อย)
            if random.random() < mutation_rate:
                mutation_idx = np.random.randint(num_phases)
                child1[mutation_idx] += np.random.randint(-2, 3)  # ปรับขึ้นลงเล็กน้อย
                child2[mutation_idx] += np.random.randint(-2, 3)

            # ปรับค่าให้รวมกันเป็น 89
            child1 = np.round((child1 / child1.sum()) * CYCLE_TIME).astype(int)
            child1[-1] = CYCLE_TIME - child1[:-1].sum()

            child2 = np.round((child2 / child2.sum()) * CYCLE_TIME).astype(int)
            child2[-1] = CYCLE_TIME - child2[:-1].sum()

            new_population.extend([child1, child2])

        population = np.array(new_population)
        generation += 1

    print("\n✅ Optimal Green Times Found:")
    print(f"🔹 Green Times: {best_solution}")
    print(f"🔹 Fitness Score: {best_fitness:.6f}")

    return best_solution

# เรียกใช้งาน Genetic Algorithm
best_green_times = genetic_algorithm()

Generation 0: Best Green Times = [43 30 16], Fitness = 0.014988
Generation 1: Best Green Times = [28 40 21], Fitness = 0.015171
Generation 2: Best Green Times = [33 29 27], Fitness = 0.015343
Generation 3: Best Green Times = [33 29 27], Fitness = 0.015343
Generation 4: Best Green Times = [33 29 27], Fitness = 0.015343
Generation 5: Best Green Times = [32 30 27], Fitness = 0.015350
Generation 6: Best Green Times = [31 29 29], Fitness = 0.015361
Generation 7: Best Green Times = [31 29 29], Fitness = 0.015361
Generation 8: Best Green Times = [31 29 29], Fitness = 0.015361
Generation 9: Best Green Times = [30 29 30], Fitness = 0.015364
Generation 10: Best Green Times = [30 29 30], Fitness = 0.015364
Generation 11: Best Green Times = [30 29 30], Fitness = 0.015364
Generation 12: Best Green Times = [30 29 30], Fitness = 0.015364
Generation 13: Best Green Times = [30 29 30], Fitness = 0.015364
Generation 14: Best Green Times = [30 29 30], Fitness = 0.015364
Generation 15: Best Green Times = [

In [77]:
import numpy as np
import random

# ค่าคงที่
CYCLE_TIME = 89  # รอบเวลาทั้งหมด
q_values = [1802, 1497, 542]  # อัตราการไหลของรถแต่ละเฟส (คัน/ชั่วโมง)
x_values = [1.062, 0.885, 0.325]  # Degree of Saturation

# ฟังก์ชันคำนวณค่าเฉลี่ยเวลารอคอยของทั้ง 3 เฟส
def calculate_wait_time(green_times):
    total_wait_time = 0
    for i in range(3):
        g_i = green_times[i]
        q_i = q_values[i]
        x_i = x_values[i]

        term1 = CYCLE_TIME  * ((1 - (g_i / CYCLE_TIME)) ** 2) / (1 - (g_i / CYCLE_TIME) * x_i)
        term2 = (x_i ** 2) / (2 * q_i * (1 - x_i))
        term3 = ((CYCLE_TIME / q_i) ** 0.5) * (x_i ** (2 + (5 * g_i / CYCLE_TIME)))

        d_i = term1 + term2 - term3
        total_wait_time += d_i

    return total_wait_time / 3  # ค่าเฉลี่ยของทั้ง 3 เฟส

# ฟังก์ชัน Fitness (ค่าฟิตเนส = ค่าเฉลี่ยรอคอย)
def fitness_function(green_times):
    return calculate_wait_time(green_times)

# ฟังก์ชันสร้างประชากรเริ่มต้น
def initialize_population(pop_size=10, num_phases=3):
    population = []
    for _ in range(pop_size):
        green_times = np.random.randint(5, CYCLE_TIME - 10, num_phases)  # สุ่มค่า
        green_times = (green_times / green_times.sum()) * CYCLE_TIME  # ปรับให้รวมกันเป็น 89
        green_times = np.round(green_times).astype(int)
        green_times[-1] = CYCLE_TIME - green_times[:-1].sum()
        population.append(green_times)
    return np.array(population)

# Genetic Algorithm (GA)
def genetic_algorithm(num_phases=3, pop_size=10, mutation_rate=0.1, max_stagnant_generations=50):
    population = initialize_population(pop_size, num_phases)

    best_wait_time = float('inf')
    best_solution = None
    stagnant_count = 0  # ตัวนับจำนวนรอบที่ค่าดีที่สุดซ้ำกัน
    best_wait_times = []  # เก็บค่าที่ดีที่สุดของแต่ละรอบ

    generation = 0
    while stagnant_count < max_stagnant_generations:
        fitness_scores = np.array([fitness_function(ind) for ind in population])
        best_idx = np.argmin(fitness_scores)
        best_current_wait_time = fitness_scores[best_idx]
        best_current_solution = population[best_idx]

        print(f"Generation {generation}: Best Green Times = {best_current_solution}, Avg Wait Time = {best_current_wait_time:.6f}")

        # เช็คค่าที่ดีที่สุด ถ้าไม่มีการเปลี่ยนแปลงก็เพิ่มตัวนับ
        if best_current_wait_time < best_wait_time:
            best_wait_time = best_current_wait_time
            best_solution = best_current_solution
            stagnant_count = 0  # รีเซ็ตตัวนับ
        else:
            stagnant_count += 1

        # เก็บค่าที่ดีที่สุดของแต่ละรอบ
        best_wait_times.append(best_current_wait_time)

        # ถ้ามีค่าเฉลี่ยรอคอยน้อยที่สุดก็หยุดทำงาน
        if best_wait_time <= min(best_wait_times):
            print("\n✅ พบค่ารอคอยเฉลี่ยต่ำสุด! หยุดการทำงาน")
            break

        # สร้างประชากรใหม่ (เลือกพ่อแม่ + crossover)
        new_population = []
        for _ in range(pop_size // 2):
            parent1 = population[np.argmin(fitness_scores)]  # ตัวที่ดีที่สุด
            parent2 = population[np.random.randint(len(population))]
            child1, child2 = (parent1 + parent2) // 2, (parent2 + parent1) // 2

            # Mutation (สุ่มเปลี่ยนแปลงเล็กน้อย)
            if random.random() < mutation_rate:
                mutation_idx = np.random.randint(num_phases)
                child1[mutation_idx] += np.random.randint(-2, 3)
                child2[mutation_idx] += np.random.randint(-2, 3)

            # ปรับค่าให้รวมกันเป็น 89
            child1 = np.round((child1 / child1.sum()) * CYCLE_TIME).astype(int)
            child1[-1] = CYCLE_TIME - child1[:-1].sum()

            child2 = np.round((child2 / child2.sum()) * CYCLE_TIME).astype(int)
            child2[-1] = CYCLE_TIME - child2[:-1].sum()

            new_population.extend([child1, child2])

        population = np.array(new_population)
        generation += 1

    print("\n✅ Optimal Green Times Found:")
    print(f"🔹 Green Times: {best_solution}")
    print(f"🔹 Avg Wait Time: {best_wait_time:.6f}")

    return best_solution

# เรียกใช้งาน Genetic Algorithm
best_green_times = genetic_algorithm()

Generation 0: Best Green Times = [ 7 36 46], Avg Wait Time = 52.051699

✅ พบค่ารอคอยเฉลี่ยต่ำสุด! หยุดการทำงาน

✅ Optimal Green Times Found:
🔹 Green Times: [ 7 36 46]
🔹 Avg Wait Time: 52.051699


In [24]:
  term1 =(89  * ((1 - (35 / 89)) ** 2)) / (1 - (35 / 89) * 1.062)
  term2 = (1.062** 2) / (2 * 1802 * (1 - 1.062))
  term3 = ((89  / 1802) ** 0.5) * (1.062 ** (2 + (5 * 35 /89)))

  d_i = term1 + term2 - term3
  d_i

56.2597666040427

In [78]:
import numpy as np
import random

# ค่าคงที่
CYCLE_TIME = 89  # รอบเวลาทั้งหมด
q_values = [1802, 1497, 542]  # อัตราการไหลของรถแต่ละเฟส (คัน/ชั่วโมง)
x_values = [0.263, 0.233, 0.17]  # Degree of Saturation

# ฟังก์ชันคำนวณค่าเฉลี่ยเวลารอคอยของทั้ง 3 เฟส
def calculate_wait_time(green_times):
    total_wait_time = 0
    for i in range(3):
        g_i = green_times[i]
        q_i = q_values[i]
        x_i = x_values[i]

        term1 = CYCLE_TIME * ((1 - (g_i / CYCLE_TIME)) ** 2) / (1 - (g_i / CYCLE_TIME) * x_i)
        term2 = (x_i ** 2) / (2 * q_i * (1 - x_i))
        term3 = ((CYCLE_TIME / q_i) ** 0.5) * (x_i ** (2 + (5 * g_i / CYCLE_TIME)))

        d_i = term1 + term2 - term3
        total_wait_time += d_i

    return total_wait_time / 3  # ค่าเฉลี่ยของทั้ง 3 เฟส

# ฟังก์ชัน Fitness (ค่าฟิตเนส = ค่าเฉลี่ยรอคอยของทั้ง 3 เฟส)
def fitness_function(green_times):
    return calculate_wait_time(green_times)

# ฟังก์ชันสร้างประชากรเริ่มต้น
def initialize_population(pop_size=10, num_phases=3):
    population = []
    for _ in range(pop_size):
        green_times = np.random.randint(5, CYCLE_TIME - 10, num_phases)  # สุ่มค่า
        green_times = (green_times / green_times.sum()) * CYCLE_TIME  # ปรับให้รวมกันเป็น 89
        green_times = np.round(green_times).astype(int)
        green_times[-1] = CYCLE_TIME - green_times[:-1].sum()
        population.append(green_times)
    return np.array(population)

# Genetic Algorithm (GA)
def genetic_algorithm(num_phases=3, pop_size=10, mutation_rate=0.1, max_generations=1000):
    population = initialize_population(pop_size, num_phases)

    best_wait_time = float('inf')
    best_solution = None
    best_generation = 0

    for generation in range(max_generations):
        fitness_scores = np.array([fitness_function(ind) for ind in population])
        best_idx = np.argmin(fitness_scores)
        best_current_wait_time = fitness_scores[best_idx]
        best_current_solution = population[best_idx]

        print(f"Generation {generation}: Green Times = {best_current_solution}, Avg Wait Time = {best_current_wait_time:.6f}")

        if best_current_wait_time < best_wait_time:
            best_wait_time = best_current_wait_time
            best_solution = best_current_solution
            best_generation = generation

        # สร้างประชากรใหม่ (เลือกพ่อแม่ + crossover)
        new_population = []
        for _ in range(pop_size // 2):
            parent1 = population[np.argmin(fitness_scores)]  # ตัวที่ดีที่สุด
            parent2 = population[np.random.randint(len(population))]
            child1, child2 = (parent1 + parent2) // 2, (parent2 + parent1) // 2

            # Mutation (สุ่มเปลี่ยนแปลงเล็กน้อย)
            if random.random() < mutation_rate:
                mutation_idx = np.random.randint(num_phases)
                child1[mutation_idx] += np.random.randint(-2, 3)
                child2[mutation_idx] += np.random.randint(-2, 3)

            # ปรับค่าให้รวมกันเป็น 89
            child1 = np.round((child1 / child1.sum()) * CYCLE_TIME).astype(int)
            child1[-1] = CYCLE_TIME - child1[:-1].sum()

            child2 = np.round((child2 / child2.sum()) * CYCLE_TIME).astype(int)
            child2[-1] = CYCLE_TIME - child2[:-1].sum()

            new_population.extend([child1, child2])

        population = np.array(new_population)

    print("\n✅ Optimal Green Times Found:")
    print(f"🔹 Generation: {best_generation}")
    print(f"🔹 Green Times: {best_solution}")
    print(f"🔹 Avg Wait Time: {best_wait_time:.6f}")

    return best_solution

# เรียกใช้งาน Genetic Algorithm
best_green_times = genetic_algorithm()

Generation 0: Green Times = [32 32 25], Avg Wait Time = 42.829611
Generation 1: Green Times = [29 27 33], Avg Wait Time = 42.773264
Generation 2: Green Times = [29 27 33], Avg Wait Time = 42.773264
Generation 3: Green Times = [30 28 31], Avg Wait Time = 42.735510
Generation 4: Green Times = [28 27 34], Avg Wait Time = 42.800789
Generation 5: Green Times = [28 27 34], Avg Wait Time = 42.800789
Generation 6: Green Times = [29 27 33], Avg Wait Time = 42.773264
Generation 7: Green Times = [28 29 32], Avg Wait Time = 42.743942
Generation 8: Green Times = [28 29 32], Avg Wait Time = 42.743942
Generation 9: Green Times = [28 29 32], Avg Wait Time = 42.743942
Generation 10: Green Times = [28 28 33], Avg Wait Time = 42.766463
Generation 11: Green Times = [28 28 33], Avg Wait Time = 42.766463
Generation 12: Green Times = [28 28 33], Avg Wait Time = 42.766463
Generation 13: Green Times = [28 28 33], Avg Wait Time = 42.766463
Generation 14: Green Times = [28 28 33], Avg Wait Time = 42.766463
Gener